In [ ]:
# Clone the repository and install the necessary packages
!git clone https://github.com/JakubMartinka/karlsruhe2024.git
!pip3 install mlatom
!pip3 install geometric
!pip3 install py3Dmol

In [ ]:
import mlatom as ml

# Universal ANI-1ccx MLP
ANI-1ccx is targeting CCSD(T)/CBS - let's see how well it works.

This is just for demonstration - we recommend to use UAIQM in real applications (https://xacs.xmu.edu.cn/docs/mlatom/tutorial_uaiqm.html)

In [ ]:
ani1ccx = ml.models.methods(method='ANI-1ccx')

## Example 1: Heat of formation
Let's calculate the heat of formation of vinylacetylene

In [ ]:
# initial xyz geometry
va_init = ml.molecule.from_xyz_string('''8

C             0.00000     0.00000     0.00000
C             1.33718     0.00000     0.00000
C             2.04862     1.23465     0.00000
C             2.65517     2.30065     0.00000
H             3.17326     3.20500     0.00000
H            -0.56880     0.91919     0.00000
H            -0.56652    -0.92292    -0.00000
H             1.95564    -0.90366     0.00000
''')

In [ ]:
# step 1 - optimize the geometry
geomopt = ml.simulations.optimize_geometry(model=ani1ccx, initial_molecule=va_init, program='geometric')
va_opt = geomopt.optimized_molecule
va_opt.view()

In [ ]:
# step 2 - run frequency and thermochemistry calculations
va_freq = ml.molecule.from_numpy(va_opt.get_xyz_coordinates(), va_opt.get_atomic_numbers())
ml.simulations.thermochemistry(model=ani1ccx, molecule=va_freq, program='pyscf')

In [ ]:
# Check vibration analysis
print("Mode     Frequencies     Reduced masses     Force Constants")
print("           (cm^-1)            (AMU)           (mDyne/A)")
for ii in range(len(va_freq.frequencies)):
    print("%d   %13.4f   %13.4f   %13.4f"%(ii,va_freq.frequencies[ii],va_freq.reduced_masses[ii],va_freq.force_constants[ii]))

# Check thermochemistry results
print('')
print(f"Zero-point vibrational energy: {va_freq.ZPE} Hartree")
print(f"Enthalpy at 298 K: {va_freq.H} Hartree")
print(f"Gibbs Free energy at 298 K: {va_freq.G} Hartree")
print(f"Heat of formation at 298 K: {va_freq.DeltaHf298 * ml.constants.Hartree2kcalpermol} kcal/mol")

In [ ]:
# check the uncertainty (UQ threshold is 1.68 kcal/mol)
va_freq.ani1ccx.energy_standard_deviation * ml.constants.Hartree2kcalpermol

In [ ]:
# let's view the first vibrational normal mode
va_freq.view(normal_mode=0)

## Example 2: MD
Let's do MD of the boring hydrogen molecule H2

In [ ]:
# use random initial conditions
mol = ml.data.molecule.from_xyz_string('''2

H             0.0000000000000           0.0000000000000           0.0000000000000
H             0.0000000000000           0.0000000000000           0.8000000000000
''')
init_cond_db = ml.generate_initial_conditions(molecule=mol,
                                                generation_method='random',
                                                initial_temperature=300,           # set initial temperature in K,
                                                number_of_initial_conditions=1     # we just want to run a single trajectory
                                             )
init_mol = init_cond_db[0]

In [ ]:
# Initializing thermostat
nose_hoover = ml.md.Nose_Hoover_thermostat(temperature=300, molecule=init_mol)

# Run molecular dynamics
dyn = ml.md(model=ani1ccx,
            molecule_with_initial_conditions=init_mol,
            thermostat=nose_hoover,
            ensemble='NVT',
            time_step=0.5, # fs
            maximum_propagation_time=100.0 # fs
           )

# Dump trajectory
traj = dyn.molecular_trajectory
traj.dump(filename='traj', format='plain_text')
traj.dump(filename='traj.h5', format='h5md')

print(f"Number of steps in the trajectory: {len(traj.steps)}")

In [ ]:
traj.view()

In [ ]:
for step in traj.steps:
    print(f'{step.step:3d}  {step.time:8.2f}   {step.molecule.internuclear_distance(0, 1):8.2f} {step.molecule.atoms[1].xyz_coordinates[2] - step.molecule.atoms[0].xyz_coordinates[2]:8.2f}')